#Install and Import Libraries

In [ ]:
!pip install --quiet optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.7 MB/s eta 0:00:00


In [ ]:
#!pip install datasets --quiet

In [ ]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
import math
from torch.utils.data import DataLoader, Dataset
import optuna
import matplotlib.pyplot as plt

In [ ]:
#from datasets import Dataset

In [ ]:
!pip install kaggle --quiet
import json
import os

In [ ]:
from google.colab import userdata
#Load kaggle credentials
kaggle_credentials = {"username":userdata.get('username'),
                      "key":userdata.get('key')}

In [ ]:
# Create the .kaggle directory
os.makedirs('/root/.kaggle', exist_ok=True)

# Save the credentials to kaggle.json
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump(kaggle_credentials, f)

# Set the permissions for the kaggle.json file
os.chmod('/root/.kaggle/kaggle.json', 0o600)

#Process Data

##Load Data

In [ ]:
!kaggle datasets download -d rmisra/news-category-dataset

Dataset URL: https://www.kaggle.com/datasets/rmisra/news-category-dataset
License(s): Attribution 4.0 International (CC BY 4.0)
 34% 9.00M/26.5M [00:00<00:00, 23.2MB/s]
100% 26.5M/26.5M [00:00<00:00, 61.5MB/s]


In [ ]:
# Unzip the downloaded file
import zipfile

with zipfile.ZipFile('news-category-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall()


In [ ]:
#Load the dataset into a pandas DataFrame
df = pd.read_json('News_Category_Dataset_v3.json', lines =True)

##Process Dataframe

In [ ]:
category_map = {category: idx for idx, category in enumerate(df['category'].unique())}

In [ ]:
df['category_number'] = df['category'].map(category_map)
df['text'] = df['headline'] + "Written by: " +df['authors'] + ' ' + df['short_description']

In [ ]:
df.head()

,link,headline,category,short_description,authors,date,category_number,text
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23,0,Over 4 Million Americans Roll Up Sleeves For O...
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23,0,"American Airlines Flyer Charged, Banned For Li..."
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23,1,23 Of The Funniest Tweets About Cats And Dogs ...
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23,2,The Funniest Tweets From Parents This Week (Se...
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22,0,Woman Who Called Cops On Black Bird-Watcher Lo...


In [ ]:
train_data, temp_data = train_test_split(df, test_size=0.2)
val_data, test_data = train_test_split(temp_data, test_size=0.5)

In [ ]:
def closest_power_of_two(n):
    return 2 ** int(math.log(n, 2))

text_lengths = df['text'].apply(len)
max_length = text_lengths.max()
MAX_LENGTH = closest_power_of_two(max_length)
print(f"Max length: {MAX_LENGTH}")

Max length: 1024


##Tokenize

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize(text):
  return tokenizer(text, padding='max_length', truncation=True, return_tensors="pt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
train_data['input_ids'] = train_data['text'].apply(lambda x: tokenize(x)['input_ids'][0])
val_data['input_ids'] = val_data['text'].apply(lambda x: tokenize(x)['input_ids'][0])
test_data['input_ids'] = test_data['text'].apply(lambda x: tokenize(x)['input_ids'][0])

In [ ]:
print(f"Train data size: {train_data.shape}")
print(f"Validation data shape: {val_data.shape}")
print(f"Test data shape: {test_data.shape}")

Train data size: (167621, 9)
Validation data shape: (20953, 9)
Test data shape: (20953, 9)


In [ ]:
BATCH_SIZE = 8
NUM_WORKERS = 2

In [ ]:
print(type(train_data['input_ids']))

<class 'pandas.core.series.Series'>


In [ ]:
print("Train data - input_ids shape:", train_data['input_ids'].shape)
print("Train data - input_ids type:", type(train_data['input_ids'][0]))
print("Train data - category_number shape:", train_data['category_number'].shape)
print("Train data - category_number type:", type(train_data['category_number'][0]))


Train data - input_ids shape: (167621,)
Train data - input_ids type: <class 'torch.Tensor'>
Train data - category_number shape: (167621,)
Train data - category_number type: <class 'numpy.int64'>


In [ ]:
class NewsDataset(Dataset):
    def __init__(self, data):
        self.text = data['input_ids'].tolist()
        self.labels = data['category_number'].tolist()

    def __getitem__(self, index):
        return self.text[index], self.labels[index]

    def __len__(self):
        return len(self.labels)

test_dataset = NewsDataset(test_data)
val_dataset = NewsDataset(val_data)
train_dataset = NewsDataset(train_data)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

#Model Class

In [ ]:
class RNNModel(nn.Module):
  def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
    super(RNNModel, self).__init__()

    #Embeding layer: Coverts input token IDs to dense vectors
    self.embedding = nn.Embedding(vocab_size, embed_dim)

    #RNN layer: LSTM (Long short term memory)
    self.rnn = nn.LSTM(embed_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True)

    #Fully connected layer: Maps the hidden state of the last time step to the output
    self.fc = nn.Linear(hidden_dim * (2 if bidirectional else 1), output_dim)

    #Dropout layer: Prevents overfitting
    self.dropout = nn.Dropout(dropout)

  def forward(self, text):
    # Text: input tensor of shape (batch_size, seq_length)

    # Embedding layer: input tensor of shape (batch_size, seq_length, embed_dim)
    embedded = self.dropout(self.embedding(text))

    # RNN: Pass emebed text though RNN: output shape  (batch_size, seq_length, hidden_dim)
    packed_output, (hidden, cell) = self.rnn(embedded)

    #If bidirectional, concatinate final forward and backward hidden states
    if self.rnn.bidirectional:
      hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)

    else:
      hidden = hidden[-1,:,:]

    # Pass hidden state through fully connected layer:shape (batch_size, output_dim)
    return self.fc(hidden)


#Train, Validate, Test

##Train Function

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
def train_model(model, dataloader, optimizer, criterion, device):
  model.train()
  total_loss, total_accurancy = 0.0, 0.0

  for texts, labels in dataloader:
    texts = texts.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    outputs = model(texts).squeeze(1)
    loss = criterion(outputs, labels)

    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    total_accurancy += (outputs.argmax(dim=1) == labels).sum().item()

  return total_loss / len(dataloader), total_accurancy / len(dataloader.dataset)

##Evalute Function

In [ ]:
def evaluate_model(model, dataloader, criterion, device):
  model.eval()
  total_loss, total_accurancy = 0.0, 0.0

  with torch.no_grad():
    for texts, labels in dataloader:
      texts = texts.to(device)
      labels = labels.to(device)

      outputs = model(texts).squeeze(1)
      loss = criterion(outputs, labels)

      total_loss += loss.item()
      total_accurancy += (outputs.argmax(dim=1) == labels).sum().item()

  return total_loss / len(dataloader), total_accurancy / len(dataloader.dataset)

##Fine Tuning

In [ ]:
def objective(trail):
  embed_dim = trail.suggest_int('embed_dim', 100, 160)
  hidden_dim = trail.suggest_int('hidden_dim', 128, 256)
  n_layers = trail.suggest_int('n_layers', 1, 2)

  dropout = trail.suggest_float('dropout', 0.1, 0.5)

  if n_layers == 1:
        dropout = 0.0

  model = RNNModel(vocab_size=len(tokenizer.vocab), embed_dim=embed_dim, hidden_dim=hidden_dim,
                   output_dim=len(category_map), n_layers=n_layers, bidirectional=False, dropout=dropout)

  model = model.to(device)

  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

  num_epochs = 10
  for epoch in range(num_epochs):
    train_loss, train_acc = train_model(model, train_dataloader, optimizer, criterion, device)
    val_loss, val_acc = evaluate_model(model, val_dataloader, criterion, device)

    return val_loss

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=3)

print('Best trial:')
trial = study.best_trial
print(f'Value: {trial.value}')
print('  Params: ')
for key, value in trial.params.items():
    print(f'{key}: {value}')


#Model

In [ ]:
best_parms = trial.params

with open('best_params.json', 'w') as f:
    json.dump(best_parms, f)

print('Best parameters saved to best_params.json')

NameError: name 'trial' is not defined

In [ ]:
with open('best_params.json', 'r') as json_file:
    best_params = json.load(json_file)

best_params

{'embed_dim': 158,
 'hidden_dim': 167,
 'n_layers': 2,
 'dropout': 0.30785018263397657}

In [ ]:
model = RNNModel(vocab_size=len(tokenizer.vocab), embed_dim=best_params['embed_dim'], hidden_dim=best_params['hidden_dim'],
                 output_dim=len(category_map), n_layers=best_params['n_layers'], bidirectional=False, dropout=best_params['dropout'])

model = model.to(device)

##Training Loop

In [ ]:
num_epochs = 50

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_losses_list = []
train_accuracies_list = []
val_losses_list = []
val_accuracies_list = []

for epoch in range(num_epochs):
  train_loss, train_acc = train_model(model, train_dataloader, optimizer, criterion, device)
  val_loss, val_acc = evaluate_model(model, val_dataloader, criterion, device)
  train_losses_list.append(train_loss)
  train_accuracies_list.append(train_acc)
  val_losses_list.append(val_loss)
  val_accuracies_list.append(val_acc)
  if epoch+1 % 10 == 0:
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')


###Plot

In [ ]:
# Plotting the training and validation metrics
epochs = np.arange(1, num_epochs + 1)
plt.figure(figsize=(12, 4))

# Plotting Losses
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses_list, label='Train')
plt.plot(epochs, val_losses_list, label='Validation')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plotting Accuracies
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies_list, label='Train')
plt.plot(epochs, val_accuracies_list, label='Validation')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

##Testing

In [ ]:
test_loss, test_acc = evaluate_model(model, test_dataloader, criterion, device)
print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')

##Save Model

In [ ]:
model_path = 'news_category_model.pth'

# Save the model's state_dict
torch.save(model.state_dict(), model_path)